In [39]:
# https://github.com/microsoft/PlanetaryComputerExamples/blob/main/competitions/s1floods/generate_auxiliary_input.ipynb
from dataclasses import dataclass
import os
from tempfile import TemporaryDirectory
from typing import List, Any, Dict

from shapely.geometry import box, mapping
import rasterio
from rasterio.warp import reproject, Resampling
import pyproj
from osgeo import gdal

from pystac_client import Client
import planetary_computer as pc
from pathlib import Path
import shutil
import errno

## Get the path to all training files

In [40]:
DATA_PATH = Path.cwd().parent.parent / "data" / "to_predict" / "test_features"

In [41]:
chip_paths = []
for file_name in os.listdir(DATA_PATH):
    if file_name.endswith("_vv.tif"):
        chip_paths.append(os.path.join(DATA_PATH, file_name))
print(f"{len(chip_paths)} chips found.")

32 chips found.


## Clean external data directory and prepare directories

In [42]:
DATA_PATH = Path.cwd().parent.parent / "data" / "to_predict"
for files in os.listdir(DATA_PATH):
    if(files != "test_features"):
        path = os.path.join(DATA_PATH, files)
        try:
            shutil.rmtree(path)
        except OSError:
            os.remove(path)

In [43]:
DATA_PATH = Path.cwd().parent.parent / "data" / "to_predict"
directories = ["nasadem", "jrc_extent", "jrc_occurrence", "jrc_recurrence", "jrc_seasonality", "jrc_transitions", "jrc_change"]
for directory in directories:
    if not os.path.exists(DATA_PATH / directory):
        try:
            os.makedirs(DATA_PATH / directory)
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise

## Connect to the planetary computer API

In [44]:
STAC_API = "https://planetarycomputer.microsoft.com/api/stac/v1"
catalog = Client.open(STAC_API)

## Define functions and class

In [45]:
@dataclass
class ChipInfo:
    """
    Holds information about a training chip, including geospatial info for coregistration
    """

    path: str
    prefix: str
    crs: Any
    shape: List[int]
    transform: List[float]
    bounds: rasterio.coords.BoundingBox
    footprint: Dict[str, Any]


def get_footprint(bounds, crs):
    """Gets a GeoJSON footprint (in epsg:4326) from rasterio bounds and CRS"""
    transformer = pyproj.Transformer.from_crs(crs, "epsg:4326", always_xy=True)
    minx, miny = transformer.transform(bounds.left, bounds.bottom)
    maxx, maxy = transformer.transform(bounds.right, bounds.top)
    return mapping(box(minx, miny, maxx, maxy))


def get_chip_info(chip_path):
    """Gets chip info from a GeoTIFF file"""
    with rasterio.open(chip_path) as ds:
        chip_crs = ds.crs
        chip_shape = ds.shape
        chip_transform = ds.transform
        chip_bounds = ds.bounds

    # Use the first part of the chip filename as a prefix
    prefix = os.path.basename(chip_path).split("_")[0]

    return ChipInfo(
        path=chip_path,
        prefix=prefix,
        crs=chip_crs,
        shape=chip_shape,
        transform=chip_transform,
        bounds=chip_bounds,
        footprint=get_footprint(chip_bounds, chip_crs),
    )

In [46]:
def reproject_to_chip(
    chip_info, input_path, output_path, resampling=Resampling.nearest
):
    """
    Reproject a raster at input_path to chip_info, saving to output_path.

    Use Resampling.nearest for classification rasters. Otherwise use something
    like Resampling.bilinear for continuous data.
    """
    with rasterio.open(input_path) as src:
        kwargs = src.meta.copy()
        kwargs.update(
            {
                "crs": chip_info.crs,
                "transform": chip_info.transform,
                "width": chip_info.shape[1],
                "height": chip_info.shape[0],
                "driver": "GTiff",
            }
        )

        with rasterio.open(output_path, "w", **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=chip_info.transform,
                    dst_crs=chip_info.crs,
                    resampling=Resampling.nearest,
                )

In [47]:
def write_vrt(items, asset_key, dest_path):
    """Write a VRT with hrefs extracted from a list of items for a specific asset."""
    hrefs = [pc.sign(item.assets[asset_key].href) for item in items]
    vsi_hrefs = [f"/vsicurl/{href}" for href in hrefs]
    gdal.BuildVRT(dest_path, vsi_hrefs).FlushCache()

In [48]:
def create_chip_aux_file(
    dir_, chip_info, collection_id, asset_key, file_name, resampling=Resampling.nearest
):
    """
    Write an auxiliary chip file.

    The auxiliary chip file includes chip_info for the Collection and Asset, and is
    saved in the same directory as the original chip with the given file_name.
    """
    output_path = os.path.join(
        Path.cwd().parent.parent / "data" / "to_predict" / dir_ , f"{chip_info.prefix}.tif"
    )
    search = catalog.search(collections=[collection_id], intersects=chip_info.footprint)
    items = list(search.get_items())
    with TemporaryDirectory() as tmp_dir:
        vrt_path = os.path.join(tmp_dir, "source.vrt")
        write_vrt(items, asset_key, vrt_path)
        reproject_to_chip(chip_info, vrt_path, output_path, resampling=resampling)
    return output_path

In [49]:
# Define a set of parameters to pass into create_chip_aux_file    
aux_file_params = [
    ("nasadem", "nasadem", "elevation", "nasadem.tif", Resampling.bilinear),
    ("jrc_extent", "jrc-gsw", "extent", "jrc-gsw-extent.tif", Resampling.nearest),
    ("jrc_occurrence", "jrc-gsw", "occurrence", "jrc-gsw-occurrence.tif", Resampling.nearest),
    ("jrc_recurrence", "jrc-gsw", "recurrence", "jrc-gsw-recurrence.tif", Resampling.nearest),
    ("jrc_seasonality", "jrc-gsw", "seasonality", "jrc-gsw-seasonality.tif", Resampling.nearest),
    ("jrc_transitions", "jrc-gsw", "transitions", "jrc-gsw-transitions.tif", Resampling.nearest),
    ("jrc_change", "jrc-gsw", "change", "jrc-gsw-change.tif", Resampling.nearest),
]

In [50]:
# Iterate over the chips and generate all aux input files.
count = len(chip_paths)
for i, chip_path in enumerate(chip_paths):
    print(f"({i+1} of {count}) {chip_path}")
    chip_info = get_chip_info(chip_path)
    for dir_, collection_id, asset_key, file_name, resampling_method in aux_file_params:
        print(f"  ... Creating chip data for {collection_id} {asset_key}")
        create_chip_aux_file(
            dir_, chip_info, collection_id, asset_key, file_name, resampling=resampling_method
        )

(1 of 32) C:\Users\maxlu\Desktop\Machine_Learning\PROJECTS\DRIVENDATA\floodwater_competition\data\to_predict\test_features\wvy00_vv.tif
  ... Creating chip data for nasadem elevation
  ... Creating chip data for jrc-gsw extent
  ... Creating chip data for jrc-gsw occurrence
  ... Creating chip data for jrc-gsw recurrence
  ... Creating chip data for jrc-gsw seasonality
  ... Creating chip data for jrc-gsw transitions
  ... Creating chip data for jrc-gsw change
(2 of 32) C:\Users\maxlu\Desktop\Machine_Learning\PROJECTS\DRIVENDATA\floodwater_competition\data\to_predict\test_features\wvy01_vv.tif
  ... Creating chip data for nasadem elevation
  ... Creating chip data for jrc-gsw extent
  ... Creating chip data for jrc-gsw occurrence
  ... Creating chip data for jrc-gsw recurrence
  ... Creating chip data for jrc-gsw seasonality
  ... Creating chip data for jrc-gsw transitions
  ... Creating chip data for jrc-gsw change
(3 of 32) C:\Users\maxlu\Desktop\Machine_Learning\PROJECTS\DRIVENDATA\

  ... Creating chip data for jrc-gsw seasonality
  ... Creating chip data for jrc-gsw transitions
  ... Creating chip data for jrc-gsw change
(19 of 32) C:\Users\maxlu\Desktop\Machine_Learning\PROJECTS\DRIVENDATA\floodwater_competition\data\to_predict\test_features\wvy18_vv.tif
  ... Creating chip data for nasadem elevation
  ... Creating chip data for jrc-gsw extent
  ... Creating chip data for jrc-gsw occurrence
  ... Creating chip data for jrc-gsw recurrence
  ... Creating chip data for jrc-gsw seasonality
  ... Creating chip data for jrc-gsw transitions
  ... Creating chip data for jrc-gsw change
(20 of 32) C:\Users\maxlu\Desktop\Machine_Learning\PROJECTS\DRIVENDATA\floodwater_competition\data\to_predict\test_features\wvy19_vv.tif
  ... Creating chip data for nasadem elevation
  ... Creating chip data for jrc-gsw extent
  ... Creating chip data for jrc-gsw occurrence
  ... Creating chip data for jrc-gsw recurrence
  ... Creating chip data for jrc-gsw seasonality
  ... Creating chip